In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

FSA = []
for post in soup.find_all('td'):
    code = post.text
    if code == '':
        break
    else:
        last_chr = int(len(code))
        last_chr = last_chr - 1
        if code[last_chr] == '\n':
            code = code[:-1]
        FSA.append(code)
post_dict = {}
postcode = []
burrough = []
neighbourhood = []
while len(FSA) > 2:
    postcode.append(FSA[0])
    burrough.append(FSA[1])
    neighbourhood.append(FSA[2])
    del FSA[0:3]

post_dic = {'Postal Code':postcode, 'Burrough':burrough, 'Neighborhood':neighbourhood}

df_post = pd.DataFrame(post_dic)

df_post = df_post[df_post.Burrough != 'Not assigned']

df_post = df_post.groupby(['Postal Code', 'Burrough'])['Neighborhood'].apply(', '.join).reset_index()

df_post.loc[df_post.Neighborhood == 'Not assigned', 'Neighborhood'] = df_post.Burrough

#print(df_post)

df_post.shape

(103, 3)

In [3]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_new = pd.merge(df_post, df_geo, on='Postal Code')
df_new.head()

,Postal Code,Burrough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
address = 'Toronto, CN'

geolocater = Nominatim()
location = geolocater.geocode(address)
latitude = location.longitude
longitude = location.longitude
print('The geographical coordinate of Toronto, Cananda are {}, {}'.format(latitude, longitude))

C:\Users\cewhite\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinate of Toronto, Cananda are -79.3870871832047, -79.3870871832047


In [54]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng, burough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Burrough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, burough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_Toronto)
    
map_Toronto 

In [6]:
Toronto_west = df_new[df_new['Burrough'] == 'West Toronto'].reset_index(drop=True)
Toronto_east = df_new[df_new['Burrough'] == 'East Toronto'].reset_index(drop=True)
Toronto_down = df_new[df_new['Burrough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_central = df_new[df_new['Burrough'] == 'Central Toronto'].reset_index(drop=True)
Toro = pd.concat([Toronto_west, Toronto_east,Toronto_down,Toronto_central])
Toro.reset_index(drop=True, inplace=True)
Toro

,Postal Code,Burrough,Neighborhood,Latitude,Longitude
0,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
3,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
5,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
6,M4E,East Toronto,The Beaches,43.676357,-79.293031
7,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
8,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
9,M4M,East Toronto,Studio District,43.659526,-79.340923


In [7]:
### TORONTO NEIGHBORHOODS -- WEST, EAST, DOWNTOWN and CENTRAL

map_Toronto_neigh = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng, burough, neighborhood in zip(Toro['Latitude'], Toro['Longitude'], Toro['Burrough'], Toro['Neighborhood']):
    label = '{}, {}'.format(neighborhood, burough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_Toronto_neigh)
    
map_Toronto_neigh

In [8]:
CLIENT_ID = '3M5MD4WTFDBCTGKO2CFVH120NXIJ1R43IRBJTVU0BGGIMSUP'
CLIENT_SECRET = 'AJRZ10JWEYOTWU44WM3JC04XJKQN2Y3XB2T5DHS5JFUMOYGX'
VERSION = '20180605'

In [9]:
radius = 500
LIMIT = 100
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID,
#     CLIENT_SECRET,
#     VERSION,
#     neighborhood_latitude,
#     neighborhood_longitude,
#     radius,
#     LIMIT)

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(
            name, 
            lat,
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                            'Neighborhood Latitude', 
                            'Neighborhood Longitude', 
                            'Venue', 
                            'Venue Latitude', 
                            'Venue Longitude', 
                            'Venue Category']
    return(nearby_venues)

In [16]:
#print(Toro['Neighborhood'][1])
toro_venues = getNearbyVenues(names=Toro['Neighborhood'], latitudes=Toro['Latitude'], longitudes=Toro['Longitude'])


Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Business reply mail Processing Centre969 Eastern
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, Sou

In [17]:
print(toro_venues.shape)
print(toro_venues.head())

(1705, 7)
                   Neighborhood  Neighborhood Latitude  \
0  Dovercourt Village, Dufferin              43.669005   
1  Dovercourt Village, Dufferin              43.669005   
2  Dovercourt Village, Dufferin              43.669005   
3  Dovercourt Village, Dufferin              43.669005   
4  Dovercourt Village, Dufferin              43.669005   

   Neighborhood Longitude                            Venue  Venue Latitude  \
0              -79.442259             The Greater Good Bar       43.669409   
1              -79.442259                         Parallel       43.669516   
2              -79.442259                          FreshCo       43.667918   
3              -79.442259          Happy Bakery & Pastries       43.667050   
4              -79.442259  Planet Fitness Toronto Galleria       43.667588   

   Venue Longitude             Venue Category  
0       -79.439267                        Bar  
1       -79.438728  Middle Eastern Restaurant  
2       -79.440754          

In [18]:
toro_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,53,53,53,53,53,53
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",48,48,48,48,48,48
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [19]:
print('There are {} unique categories.'.format(len(toro_venues['Venue Category'].unique())))

There are 234 unique categories.


In [21]:
toro_onehot = pd.get_dummies(toro_venues[['Venue Category']], prefix="", prefix_sep="")
toro_onehot['Neighborhood'] = toro_venues['Neighborhood']
fixed_columns = [toro_onehot.columns[-1]] + list(toro_onehot.columns[:-1])
toro_onehot = toro_onehot[fixed_columns]
toro_onehot.head()
toro_onehot.shape

(1705, 234)

In [22]:
toro_group = toro_onehot.groupby('Neighborhood').mean().reset_index()
toro_group.head()
toro_group.shape

(38, 234)

In [24]:
num_top_venues = 5
for hood in toro_group['Neighborhood']:
    print("----"+hood+"----")
    temp = toro_group[toro_group['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', 
ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.06
2         Cheese Shop  0.04
3  Seafood Restaurant  0.04
4                Café  0.04


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.14
1                    Café  0.10
2          Breakfast Spot  0.10
3  Furniture / Home Store  0.05
4    Caribbean Restaurant  0.05


----Business reply mail Processing Centre969 Eastern----
              venue  freq
0       Yoga Studio  0.06
1  Recording Studio  0.06
2        Skate Park  0.06
3    Farmers Market  0.06
4               Spa  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  fre

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toro_group['Neighborhood']
for ind in np.arange(toro_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toro_group.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Gym,Cosmetics Shop,Bar,Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Café,Cheese Shop,Steakhouse,Beer Bar,Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Stadium,Grocery Store,Furniture / Home Store,Italian Restaurant,Falafel Restaurant,Convenience Store,Performing Arts Venue
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Butcher,Burrito Place,Recording Studio,Restaurant,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Terminal,Airport Service,Airport Lounge,Boat or Ferry,Airport Gate,Plane,Sculpture Garden,Airport Food Court,Airport,Harbor / Marina
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Bakery,Pub,Café,Pizza Place,Chinese Restaurant,Indian Restaurant,Italian Restaurant
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Falafel Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Vietnamese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Caribbean Restaurant
8,Christie,Grocery Store,Café,Park,Athletics & Sports,Convenience Store,Nightclub,Restaurant,Diner,Baby Store,Italian Restaurant
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Burger Joint,Restaurant,Pub,Café,Gastropub,Fast Food Restaurant


In [33]:
kclusters = 5
toro_group_clustering = toro_group.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toro_group_clustering)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [38]:
toro_merged = Toro
toro_merged['Cluster Labels'] = kmeans.labels_
toro_merged = toro_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toro_merged.head()

,Postal Code,Burrough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,0,Supermarket,Bakery,Café,Discount Store,Middle Eastern Restaurant,Athletics & Sports,Liquor Store,Bank,Portuguese Restaurant,Bar
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Café,Restaurant,Coffee Shop,Cocktail Bar,Asian Restaurant,Pizza Place,Bakery,Vietnamese Restaurant,French Restaurant
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,0,Coffee Shop,Café,Breakfast Spot,Stadium,Grocery Store,Furniture / Home Store,Italian Restaurant,Falafel Restaurant,Convenience Store,Performing Arts Venue
3,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,0,Bar,Mexican Restaurant,Café,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Diner,Speakeasy,Flea Market,Bakery
4,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,0,Gift Shop,Breakfast Spot,Bar,Burger Joint,Dessert Shop,Movie Theater,Cuban Restaurant,Dog Run,Restaurant,Piano Bar


In [55]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = [0]
for lat, lon, poi, cluster in zip(toro_merged['Latitude'], toro_merged['Longitude'], toro_merged['Neighborhood'], toro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [49]:
toro_merged.loc[toro_merged['Cluster Labels'] == 0, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Burrough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Toronto,0,Supermarket,Bakery,Café,Discount Store,Middle Eastern Restaurant,Athletics & Sports,Liquor Store,Bank,Portuguese Restaurant,Bar
1,West Toronto,0,Bar,Café,Restaurant,Coffee Shop,Cocktail Bar,Asian Restaurant,Pizza Place,Bakery,Vietnamese Restaurant,French Restaurant
2,West Toronto,0,Coffee Shop,Café,Breakfast Spot,Stadium,Grocery Store,Furniture / Home Store,Italian Restaurant,Falafel Restaurant,Convenience Store,Performing Arts Venue
3,West Toronto,0,Bar,Mexican Restaurant,Café,Fast Food Restaurant,Fried Chicken Joint,Sandwich Place,Diner,Speakeasy,Flea Market,Bakery
4,West Toronto,0,Gift Shop,Breakfast Spot,Bar,Burger Joint,Dessert Shop,Movie Theater,Cuban Restaurant,Dog Run,Restaurant,Piano Bar
5,West Toronto,0,Coffee Shop,Café,Italian Restaurant,Restaurant,Sushi Restaurant,Pizza Place,Burrito Place,Pub,Dessert Shop,Diner
6,East Toronto,0,Coffee Shop,Pub,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
7,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Bubble Tea Shop,Bakery,Spa,Lounge
8,East Toronto,0,Park,Gym,Hotel,Sandwich Place,Brewery,Fast Food Restaurant,Italian Restaurant,Fish & Chips Shop,Burger Joint,Steakhouse
9,East Toronto,0,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Breakfast Spot


In [50]:
toro_merged.loc[toro_merged['Cluster Labels'] == 1, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Burrough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Café,Cheese Shop,Steakhouse,Beer Bar,Restaurant
24,Downtown Toronto,1,Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Vietnamese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Caribbean Restaurant
27,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,Gym,American Restaurant,Deli / Bodega,Bar,Gastropub


In [51]:
toro_merged.loc[toro_merged['Cluster Labels'] == 2, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Burrough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Deli / Bodega,Italian Restaurant,Steakhouse,Seafood Restaurant


In [52]:
toro_merged.loc[toro_merged['Cluster Labels'] == 3, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Burrough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Downtown Toronto,3,Grocery Store,Café,Park,Athletics & Sports,Convenience Store,Nightclub,Restaurant,Diner,Baby Store,Italian Restaurant


In [53]:
toro_merged.loc[toro_merged['Cluster Labels'] == 4, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Burrough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Central Toronto,4,Garden,Ice Cream Shop,Women's Store,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
